In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import BedrockEmbeddings

BedrockEmbeddings = BedrockEmbeddings(
    region_name="us-east-1", model_id = "cohere.embed-multilingual-v3"
)

from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, BedrockEmbeddings)
retriever = vector.as_retriever()

In [3]:
retriever.get_relevant_documents("how to upload a dataset")[1]

Document(page_content='import Clientfrom langsmith.evaluation import evaluateclient = Client()# Define dataset: these are your test casesdataset_name = "Sample Dataset"dataset = client.create_dataset(dataset_name, description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0

In [5]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [47]:
# build google search tool
import os
# load from env file
from dotenv import load_dotenv
load_dotenv()

from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

search = GoogleSearchAPIWrapper(k=1)

search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

In [13]:
search_tool.run("What is Langchain?")

'LangChain is a framework for developing applications powered by large language models (LLMs).'

In [14]:
tools = [search_tool, retriever_tool]

In [38]:
from langchain_community.chat_models import BedrockChat
# aws bedrock list-foundation-models --by-output-modality TEXT --region us-east-1 | jq -r '.modelSummaries[] | select(.modelName | contains("Claude 3")) | .modelId'
BedrockChat = BedrockChat(
    region_name="us-east-1", model_id="anthropic.claude-3-sonnet-20240229-v1:0"
)
response = BedrockChat.invoke("What is Langchain?")
response.content

"LangChain is an open-source Python library designed for building applications with large language models (LLMs) like OpenAI's GPT-3, Anthropic's Claude, and others. It provides a set of abstractions and primitives that make it easier to interact with LLMs, combine them with other sources of information and computation, and build complex applications involving natural language processing (NLP) tasks.\n\nSome key features and capabilities of LangChain include:\n\n1. **Language Model Abstraction**: LangChain provides a consistent interface for interacting with different LLMs, making it easier to switch between providers or use multiple models simultaneously.\n\n2. **Data Loading and Retrieval**: It includes utilities for loading and retrieving data from various sources, such as files, databases, web pages, and APIs, which can be used to provide additional context or information to the LLM.\n\n3. **Memory and Agents**: LangChain supports the concept of memory, allowing the LLM to maintain

In [43]:
from langchain_community.llms import Bedrock

# aws bedrock list-foundation-models --by-output-modality TEXT --region us-east-1 | jq -r '.modelSummaries[] | select(.modelName | contains("Command")) | .modelId'
# error prompt when using the model_id "meta.llama2-13b-v1": The provided model doesn't support on-demand throughput, use a different model instead
BedrockLLM = Bedrock(
    region_name="us-east-1", model_id="cohere.command-text-v14"
)

response = BedrockLLM.invoke("What is Langchain?")
response

# Note current claude3/Cohere can't support agent with error: "This function requires a .bind_tools method be implemented on the LLM.", we need to implement such method e.g. including parse the function calling similar like OpenAI. 
# Futhermore other model in Bedrock can't even be executed, error prompt when using the model_id "meta.llama2-13b-v1": The provided model doesn't support on-demand throughput, use a different model instead

" LangChain is a programming language based on a blockchain architecture, developed by researchers from the University of Illinois at Urbana-Champaign. The language is designed to allow developers to create smart contracts and decentralized applications (DApps) on the blockchain. \n\nLangChain's architecture is based on a distributed ledger, which is a shared database that is accessible by multiple parties. This architecture allows for the creation of smart contracts, which are self-executing contracts that are stored on the blockchain and automatically enforced by the network. \n\nLangChain also provides a programming language that allows developers to write smart contracts, similar to how they would write code for a traditional application. The language is designed to be high-level and easy to use, allowing developers to focus on the logic of the contract rather than the low-level details of blockchain programming. \n\nThe goal of LangChain is to make it easier for developers to buil

In [49]:
from langchain_openai import ChatOpenAI

OpenAILLM = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
response = OpenAILLM.invoke("What is Langchain?")
response.content

'Langchain is a blockchain-powered language learning platform that aims to revolutionize the way people learn languages. The platform uses cutting-edge technology to provide personalized language learning experiences that are both effective and engaging. Langchain plans to leverage blockchain to create a decentralized language learning community where users can access high-quality language learning materials, interact with native speakers, and earn rewards for reaching language learning milestones.'

In [24]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [50]:
from langchain.agents import create_tool_calling_agent

# agent = create_tool_calling_agent(BedrockLLM, tools, prompt)
agent = create_tool_calling_agent(OpenAILLM, tools, prompt)

In [51]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [52]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [54]:
agent_executor.invoke({"input": "what is the exchange rate for USD to EUR in April 2024?"})



> Entering new AgentExecutor chain...



Invoking: `google_search` with `exchange rate for USD to EUR in April 2024`


US dollar (USD). ECB euro reference exchange rate. 11 April 2024. EUR 1 = USD 1.0729 -0.0131(-1.2%). Change from 11 April 2023 to 11 April 2024. Min (3 October ...The exchange rate for USD to EUR in April 2024 is 1 EUR = 1.0729 USD.

> Finished chain.


{'input': 'what is the exchange rate for USD to EUR in April 2024?',
 'output': 'The exchange rate for USD to EUR in April 2024 is 1 EUR = 1.0729 USD.'}

In [55]:
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [57]:
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)



> Entering new AgentExecutor chain...


 Your name is Bob. How can I assist you further, Bob? 

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'input': "what's my name?",
 'output': ' Your name is Bob. How can I assist you further, Bob? '}

In [63]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...


Hello, Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello, Bob! How can I assist you today?'}

In [64]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...



Invoking: `langsmith_search` with `{'query': 'What is my name'}`


Getting started with LangSmith | 🦜️🛠️ LangSmith

{ traceable } from "langsmith/traceable";import { wrapOpenAI } from "langsmith/wrappers";// Auto-trace LLM calls in-contextconst client = wrapOpenAI(new OpenAI());// Auto-trace this functionconst pipeline = traceable(async (user_input) => {    const result = await client.chat.completions.create({        messages: [{ role: "user", content: user_input }],        model: "gpt-3.5-turbo",    });    return result.choices[0].message.content;});await pipeline("Hello, world!")// Out: Hello there! How can I assist you today?View a sample output trace.Learn more about tracing on the tracing page.Create your first evaluation​Evalution requires a system to test, data to serve as test cases, and optionally evaluators to grade the results. Here we use a built-in accuracy evaluator.PythonTypeScriptfrom langsmith import Clientfrom langsmith.evaluation import evaluateclient = Client()# De

{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello, Bob! How can I assist you today?')],
 'output': "I'm sorry, I don't have access to that information. How can I assist you today?"}